In [1]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [2]:
plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']
plot_colours = ['#043ef9', '#eb5334', '#34eb37']
def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colours[i],size=12)
        ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colours[i],size=12)
        ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()


In [3]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [5]:
df_raw.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2019-01-01T22:00:00.000000000Z,2616,109.590,109.726,109.533,109.598,109.540,109.718,109.493,109.591,109.640,109.736,109.557,109.606
1,2019-01-02T02:00:00.000000000Z,2710,109.600,109.613,109.299,109.304,109.593,109.606,109.291,109.297,109.608,109.620,109.307,109.312
2,2019-01-02T06:00:00.000000000Z,6642,109.302,109.348,108.710,108.943,109.294,109.341,108.702,108.937,109.310,109.354,108.717,108.949
3,2019-01-02T10:00:00.000000000Z,6613,108.946,109.284,108.822,109.191,108.939,109.278,108.813,109.185,108.952,109.291,108.829,109.197
4,2019-01-02T14:00:00.000000000Z,7780,109.189,109.464,109.177,109.442,109.182,109.457,109.170,109.435,109.196,109.471,109.184,109.449


In [6]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [7]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()

In [8]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c
0,2019-01-01T22:00:00.000000000Z,109.590,109.726,109.533,109.598,109.591,109.606
1,2019-01-02T02:00:00.000000000Z,109.600,109.613,109.299,109.304,109.297,109.312
2,2019-01-02T06:00:00.000000000Z,109.302,109.348,108.710,108.943,108.937,108.949
3,2019-01-02T10:00:00.000000000Z,108.946,109.284,108.822,109.191,109.185,109.197
4,2019-01-02T14:00:00.000000000Z,109.189,109.464,109.177,109.442,109.435,109.449


In [9]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:#BUY
        return (row.RANGE_prev * ENTRY_PRC) + row.mid_h_prev
    elif row.SIGNAL == -1:#SELL
        return row.mid_l_prev - (row.RANGE_prev * ENTRY_PRC) 
    else:
        return 0
    
def get_stop_loss(row):
    if row.SIGNAL == 1:#BUY
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:#SELL
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0
    
def get_take_profit(row):
    if row.SIGNAL == 1:#BUY
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:#SELL
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0

In [10]:
df = df_raw[['time','mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l #hi minus low
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)

df.reset_index(drop=True, inplace=True)

In [11]:
#Buy stop order @ prev candles high + 10% of range
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
#Stop loss @ prev candles high - 40% of range
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
#Target @ prev candles high + 80% of range
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

In [12]:
#only our trades
df[df.SIGNAL!=0].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
2,2019-01-02T10:00:00.000000000Z,108.946,109.284,108.822,109.191,109.185,109.197,0.462,109.348,108.710,0.638,1,-1,-1,108.6462,108.9014,108.1358
6,2019-01-03T02:00:00.000000000Z,107.535,107.550,106.748,106.970,106.963,106.977,0.802,108.915,104.819,4.096,-1,-1,-1,104.4094,106.0478,101.1326
10,2019-01-03T18:00:00.000000000Z,108.026,108.028,107.395,107.662,107.634,107.691,0.633,108.305,107.106,1.199,-1,1,1,108.4249,107.9453,109.3841
11,2019-01-03T22:00:00.000000000Z,107.672,107.974,107.516,107.840,107.832,107.849,0.458,108.028,107.395,0.633,1,-1,-1,107.3317,107.5849,106.8253
13,2019-01-04T06:00:00.000000000Z,108.257,108.284,107.823,108.032,108.025,108.038,0.461,108.444,107.808,0.636,-1,1,1,108.5076,108.2532,109.0164


In [13]:
df[df.SIGNAL!=0].shape

(1551, 17)

In [14]:
df.shape

(8722, 17)

In [15]:
class Trade():
    def __init__(self, row):
        #These will be later columns in our table
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)    
    
    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0            
            elif row.mid_c >= self.SL:
                self.result = -1.0   
        
        if self.result is not None:
            self.running = False
            self.stopped = row.time

In [16]:
open_trades = []
closed_trades = []
for index, row in df.iterrows():
    #check any open trades and update
    for ot in open_trades:
        ot.update(row)
        #add to closed trades and remove from open
        if ot.stopped is not None: 
            closed_trades.append(ot)

    open_trades = [x for x in open_trades if x.stopped is None]
    #signal and open trade
    if row.SIGNAL !=0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))




In [17]:
#len(closed_trades)
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [18]:
df_trades.head()

,candle_date,direction,entry,TP,SL,running,result,stopped,index,opened
0,2019-01-02T10:00:00.000000000Z,-1,108.6462,108.1358,108.9014,False,2.0,2019-01-03T02:00:00.000000000Z,5,2019-01-02T22:00:00.000000000Z
1,2019-01-04T06:00:00.000000000Z,1,108.5076,109.0164,108.2532,False,-1.0,2019-01-07T02:00:00.000000000Z,16,2019-01-04T18:00:00.000000000Z
2,2019-01-17T02:00:00.000000000Z,-1,108.8072,108.5768,108.9224,False,-1.0,2019-01-17T14:00:00.000000000Z,67,2019-01-17T06:00:00.000000000Z
3,2019-01-17T22:00:00.000000000Z,1,109.4501,109.8269,109.2617,False,2.0,2019-01-18T14:00:00.000000000Z,73,2019-01-18T06:00:00.000000000Z
4,2019-01-18T10:00:00.000000000Z,1,109.6249,109.8561,109.5093,False,-1.0,2019-01-22T02:00:00.000000000Z,75,2019-01-18T14:00:00.000000000Z


In [19]:
#df[df.SIGNAL!=0].head(10)
df[df.SIGNAL!=0].to_pickle("USD_JPY_H4_trades.pkl")

In [20]:
df_trades.result.sum()

440.0

In [21]:
df_plot = df.iloc[0:60]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells =  df_plot[df_plot.SIGNAL == -1]